<a href="https://colab.research.google.com/github/geocarvalho/uni-proj/blob/master/IN1164/project_1/PSO_and_ABC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação: 
- PSO padrão e uma variação.
- ABC (ou Firefly) padrão e uma variação.


Testar em 6 funções de benchmark com variação de atributos de 10, 20 e 50.
Funções:
- [Ackley function](http://benchmarkfcns.xyz/benchmarkfcns/ackleyfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/ackley.html)
- [Alpine function](http://benchmarkfcns.xyz/benchmarkfcns/alpinen1fcn.html)
- [Schwefel Function](http://benchmarkfcns.xyz/benchmarkfcns/schwefelfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/schwefel.html)
- [Happy Cat Function](http://benchmarkfcns.xyz/benchmarkfcns/happycatfcn.html) 
- [Brown function](http://benchmarkfcns.xyz/benchmarkfcns/brownfcn.html)
- [Exponential function](http://benchmarkfcns.xyz/benchmarkfcns/exponentialfcn.html)

Para funções olhar o site: http://benchmarkfcns.xyz/

# Funções de fitness


In [1]:
import math
import random
import numpy as np

def simple_func(position):
  """ Simple function that models the problem """
  return position[0]**2 + position[1]**2 + 1

def sphere_func(positions):
  """ Sphere function 
  http://benchmarkfcns.xyz/benchmarkfcns/spherefcn.html"""
  c = np.array(positions)
  x = c**2
  return np.sum(x)

def ackley_func(positions):
  """ Ackley function"""
  c = np.array(positions)
  firstSum = np.sum(c**2.0)
  secondSum = np.sum(np.cos(2.0*math.pi*c))
  n = float(len(positions))
  return -20.0*math.exp(-0.2*math.sqrt(firstSum/n)) - math.exp(secondSum/n) + 20 + math.e

def alpine_func(positions):
  """ Alpine function """
  c = np.array(positions)
  scores = np.sum(abs(c * np.sin(c) + 0.1*c))
  return scores

def schwefel_func(positions):
  """F7 Schwefel's function
  multimodal, asymmetric, separable"""
  c = np.array(positions)
  n = len(c)
  alpha = 418.982887
  fitness = np.sum(c * np.sin(np.sqrt(abs(c))))
  return alpha * n - fitness

def happy_cat_func(positions):
  """ Happy Cat function """
  alpha = 0.5
  c = np.array(positions)
  n = len(c)
  x2 = np.sum(c*c)
  scores = (((x2 - n)**2)**alpha + (0.5*x2 + np.sum(c)))/ (n + 0.5)
  return scores

def brown_func(positions):
  """ Brown function """
  c = np.array(positions)
  n = len(c)
  x = c**2
  scores = 0
  for i in range(n-1):
    scores = scores + x[i]**(x[i+1] + 1) + x[i+1]**(x[i]+1)
  return scores

def exponential_func(positions):
  """ Exponential function """
  c = np.array(positions)
  x2 = c**2
  scores = -np.exp(-0.5 * np.sum(x2))
  return scores


# PSO padrão

* [1995 Particle swarm optimization](https://ieeexplore.ieee.org/abstract/document/488968)
* [1998 A Modified Particle Swarm Optimizer](https://ieeexplore.ieee.org/document/699146)

In [46]:
def PSO(problem, dimension, var_min, var_max, n_iterations, n_particles, 
        w, c1, c2, show_iter):
  """ PSO algorithm """
  # Inicialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = np.copy(particle_position_vector)
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.full(shape=dimension, fill_value=0)
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0
  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = np.copy(particle_position_vector[p])
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      # Check if the positions is var_min<x<var_max
      for value in new_position:
        index = list(new_position).index(value)
        new_position[index] = np.max([var_min, value])
        new_position[index] = np.min([var_max, new_position[index]])
      particle_position_vector[p] = new_position
    
    # Calculate gbest
    gbest_candidate = np.min(pbest_fitness_value)
    index_gbest = list(pbest_fitness_value).index(gbest_candidate)
    if gbest_fitness_value > gbest_candidate:
      gbest_fitness_value = gbest_candidate
      gbest_position = np.copy(pbest_position[index_gbest])
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)

    iteration += 1

  return gbest_position, gbest_fitness_value

In [60]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 50, "n_particles": 10,
              "w": 0.8, "c1": 1.5, "c2": 1.5, "show_iter": False}
    gbest_pos, gbest_value = PSO(**kwargs)
    print("dimension: %s,  %s, gbest: %s" % (
        dim, name, gbest_value))

dimension: 10,  Ackley function, gbest: 4.4651535393709025
dimension: 10,  Alpine function, gbest: 0.5057087235238676
dimension: 10,  Schwefel's function, gbest: 4172.8414280022325
dimension: 10,  Happy Cat function, gbest: 0.18565262411178585
dimension: 10,  Brown function, gbest: 0.8822726002624524
dimension: 10,  Exponential function, gbest: -0.749209113469025
dimension: 20,  Ackley function, gbest: 1.3297720103433268
dimension: 20,  Alpine function, gbest: 0.07231594251762728
dimension: 20,  Schwefel's function, gbest: 8362.264817981892
dimension: 20,  Happy Cat function, gbest: 0.3500924647481427
dimension: 20,  Brown function, gbest: 3.445086310107728
dimension: 20,  Exponential function, gbest: -0.019191816755387215
dimension: 50,  Ackley function, gbest: 2.303687092694862
dimension: 50,  Alpine function, gbest: 1.6362777041398016
dimension: 50,  Schwefel's function, gbest: 20891.600416242938
dimension: 50,  Happy Cat function, gbest: 0.6109132173414291
dimension: 50,  Brown fun

# Variação do PSO

* [2002 The particle swarm - explosion, stability, and convergence in a multidimensional complex space](https://ieeexplore.ieee.org/abstract/document/985692)

> Constriction coefficients

$\chi = \frac{2k}{|2 - \phi - \sqrt{\phi^2 - 4\phi}|}$

* Onde $0\leq k \leq 1$ e $\phi_1, \phi_2 = 2.05$;
* Valor de $k=1$;
* Substituições $w = \chi$, $c_1 = \chi.\phi_1$ e $c_2 = \chi.\phi_2$.
* Além disso, $\phi = \phi_1 + \phi_2 \geq 4$

In [57]:
def PSO_cc(problem, dimension, var_min, var_max, n_iterations, n_particles,
        wdamp, w, c1, c2, show_iter):
  """ PSO algorithm """
  # Inicialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = np.copy(particle_position_vector)
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.full(shape=dimension, fill_value=0)
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0

  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = particle_position_vector[p]
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      # Check if the positions is var_min<x<var_max
      for value in new_position:
        index = list(new_position).index(value)
        new_position[index] = np.max([var_min, value])
        new_position[index] = np.min([var_max, new_position[index]])
      particle_position_vector[p] = new_position
    
    # Calculate gbest
    gbest_candidate = np.min(pbest_fitness_value)
    index_gbest = list(pbest_fitness_value).index(gbest_candidate)
    if gbest_fitness_value > gbest_candidate:
      gbest_fitness_value = gbest_candidate
      gbest_position = np.copy(pbest_position[index_gbest])
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)

    iteration += 1
    w *= wdamp

  # print("The best position is: ", gbest_position, " with value of ", gbest_fitness_value,
  #       " in iteration number ", iteration)
  return gbest_position, gbest_fitness_value

In [59]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

phi1 = 2.05
phi2 = 2.05
phi = phi1 + phi2
kappa = 1
chi = 2*kappa/(abs(2-phi-math.sqrt(phi**2 - 4*phi)))

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": 0, 
              "var_max": 10, "n_iterations": 100, "n_particles": 50,
              "wdamp": 1, "w": chi, "c1": chi*phi1, "c2": chi*phi2, "show_iter": False}
    gbest_pos, gbest_value = PSO_cc(**kwargs)
    print("For dimension %s using the %s, the gbest was: %s" % (dim, name, gbest_value))

For dimension 10 using the Ackley function, the gbest was: 4.440892098500626e-16
For dimension 10 using the Alpine function, the gbest was: 0.0
For dimension 10 using the Schwefel's function, the gbest was: 4150.976685224504
For dimension 10 using the Happy Cat function, the gbest was: 0.9523809523809523
For dimension 10 using the Brown function, the gbest was: 0.0
For dimension 10 using the Exponential function, the gbest was: -1.0
For dimension 20 using the Ackley function, the gbest was: 4.440892098500626e-16
For dimension 20 using the Alpine function, the gbest was: 0.0
For dimension 20 using the Schwefel's function, the gbest was: 8302.822728619045
For dimension 20 using the Happy Cat function, the gbest was: 0.975609756097561
For dimension 20 using the Brown function, the gbest was: 0.0
For dimension 20 using the Exponential function, the gbest was: -1.0
For dimension 50 using the Ackley function, the gbest was: 4.440892098500626e-16
For dimension 50 using the Alpine function, th

# ABC padrão

In [ ]:
def fitness_calc(value):
  """ Calculate the fitness based on the object function result """
  if value >= 0:
    return 1/(1+value)
  else:
    return 1 + abs(value)

def generate_new_solution(dimension, food_source, num, food, fit, 
                          var_min, var_max, trial, fitness_func,
                          population, obj):
  """ Generates a new solution """
  # Phi is a random number between -1 and 1
  phi = random.uniform(-1,1)
  ## Select a random variable to change
  var_index_lst = list(range(dimension))
  var_index = random.choice(var_index_lst)
  food_var = food[var_index]
  ## Select a random partner
  partner_index_lst = list(range(food_source))
  partner_index_lst.remove(num)
  partner_index = random.choice(partner_index_lst)
  partner = population[partner_index]
  partner_var = partner[var_index]
  ## Create a new food location
  x_new = food_var + (phi*(food_var - partner_var))
  ## Check if the new location violates the lower bound (0<=x<=10)
  x_new = np.max([var_min, x_new])
  x_new = np.min([var_max, x_new])
  copy_food = np.copy(food)
  copy_food[var_index] = x_new
  ## Evaluate the fitness
  new_obj = fitness_func(copy_food)
  new_fit = fitness_calc(new_obj)
  ## Perform greedy selection
  if new_fit > fit[num]:
    food[var_index] = x_new
    fit[num] = new_fit
    obj[num] = new_obj
    trial[num] = 0
  else:
    trial[num] += 1
  return food, obj, fit, trial

def ABC(dimension, fitness_func, var_min, var_max, pop_size, limit,
        n_cycles, show_iter):
  # Parameters
  employed_bees = int(pop_size/2)
  onlooker_bees = int(pop_size/2)
  food_source = int(pop_size/2)

  # Initialization
  ## The food source population with 4 variables, create random value 0<x<10
  population = np.random.uniform(var_min,var_max,(food_source, dimension))
  ## Calculate the object function for each food source
  obj = np.apply_along_axis(fitness_func, 1, population)
  ## Calculate randintfitness of the population
  fit = np.array([fitness_calc(value) for value in obj])
  ## Generate initial trial vector
  trial = np.zeros(food_source)
  ## Start the best solution variable
  b_fit_solution = np.max(fit)
  b_index = list(fit).index(b_fit_solution)
  b_food = population[b_index]
  b_obj = obj[b_index]
  b_cycle = 0
  for cycle in range(n_cycles):
    # Employed bee phase
    for num1, food in enumerate(population, start=0):
      food, obj, fit, trial = generate_new_solution(
          dimension, food_source, num1, food, fit, var_min, var_max,
          trial, fitness_func, population, obj)
    # Calculate the probability values
    max = np.max(fit)
    prob = ((fit/max)*0.9)+0.1
    # Onlooker bee phase
    num2 = 0
    for obee in range(onlooker_bees):
      search_for_food = True
      while search_for_food:
        # Control the flow to search for food using num and food
        if num2 == len(population-1):
          num2 = 0
        food = population[num2]
        ## Select a random number
        r = random.uniform(0,1)
        if r < prob[num2]:
          food, obj, fit, trial = generate_new_solution(
              dimension, food_source, num2, food, fit, var_min, var_max,
              trial, fitness_func, population, obj)
          num2 += 1
          search_for_food = False
        else:
          num2 += 1
    # Memorize the best solution
    candidate = np.max(fit)
    if candidate > b_fit_solution:
      b_index = list(fit).index(candidate)
      b_food = np.copy(population[b_index])
      b_fit_solution = fit[b_index]
      b_obj = obj[b_index]
      b_cycle = cycle
      if show_iter:
        print("____Inter values")
        print(population)
        print(obj)
        print(fit)
        print(trial)
    # Scout bee phase
    ## Select one solution for which the trial is greater than limit
    trial_val = np.max(trial)
    num3 = list(trial).index(trial_val)
    if trial_val > limit:
      population[num3] = np.random.uniform(var_min,var_max,(dimension))
      obj[num3] = fitness_func(population[num3])
      fit[num3] = fitness_calc(obj[num3])
      trial[num3] = 0
  if show_iter:
    print("\n____Final values")
    print(population)
    print(obj)
    print(fit)
    print(trial)
  return b_food, b_obj, b_fit_solution, b_cycle


In [ ]:
# ABC Problem

fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"dimension": dim, "fitness_func": func, "var_min": 0.0, 
              "var_max": 10.0, "pop_size": 10, "limit": 5,
              "n_cycles": 50, "show_iter": False}
    b_food, b_obj, b_fit_solution, b_cycle = ABC(**kwargs)
    print("dimension: %s, %s, object: %s, fitness: %s, cycle: %s" % (
            dim, name, b_obj, b_fit_solution, b_cycle))

# print("\n____Best solution:")
# print("best positions: ", b_food)
# print("best object: ", b_obj)
# print("best fitness: ",b_fit_solution)
# print("best cycle: ", b_cycle)

dimension: 10, Ackley function, object: 6.917663827293989, fitness: 0.1262998811029047, cycle: 33
dimension: 10, Alpine function, object: 1.8848250302256184, fitness: 0.34664147375405685, cycle: 25
dimension: 10, Schwefel's function, object: 4151.147983421825, fitness: 0.00024083920033502522, cycle: 48
dimension: 10, Happy Cat function, object: 0.9864819660656426, fitness: 0.5034025060799144, cycle: 40
dimension: 10, Brown function, object: 1145.2640423749242, fitness: 0.0008723993452050695, cycle: 30
dimension: 10, Exponential function, object: -1.3360302162641798e-60, fitness: 1.0, cycle: 0
dimension: 20, Ackley function, object: 10.67964310301392, fitness: 0.08561905455329805, cycle: 48
dimension: 20, Alpine function, object: 22.162156167390883, fitness: 0.04317387348453603, cycle: 47
dimension: 20, Schwefel's function, object: 8304.01609972039, fitness: 0.00012040915851248834, cycle: 49
dimension: 20, Happy Cat function, object: 12.547206612706432, fitness: 0.07381595546509659, cyc

# Outras referências

* [A novel particle swarm optimization algorithm with adaptive inertia weight](https://www.sciencedirect.com/science/article/abs/pii/S156849461100055X)
* [The particle swarm optimization algorithm: convergence analysis and parameter selection](https://www.sciencedirect.com/science/article/abs/pii/S0020019002004477)
* [An Overview of Particle Swarm Optimization Variants](https://www.sciencedirect.com/science/article/pii/S1877705813001823)
* [2020 Population size in Particle Swarm Optimization](https://www.sciencedirect.com/science/article/pii/S2210650220303710)
* [Lec 10 : Particle Swarm Optimization](https://www.youtube.com/watch?v=UKx76ThMXDM)
* [Lec 11 : Implementation of Particle Swarm Optimization using MATLAB](https://www.youtube.com/watch?v=2VJl37RWvkw)
* [Lec 18 : Working of Artificial Bee Colony Algorithm](https://youtu.be/MAhlPwK4_fI)
* [Lec 19 : Implementation of Artificial Bee Colony using MATLAB](https://www.youtube.com/watch?v=2Huy72h7Y20)